# Topic Modeling Data Preparation

reference:
* [Beginners Guide to Topic Modeling in Python](https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/)


* data format?

In [1]:
import pandas as pd
import numpy as np
import os

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

from pathlib import Path

from sklearn import preprocessing

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

import gensim

Folder "C:\Users\raymo\AppData\Local\Temp" will be used to save temporary dictionary and corpus.


C:\Users\raymo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-11-16 12:02:56,210 : INFO : 'pattern' package not found; tag filters are not available for English


#### To load entire dataset

#### To load only field of interest

## Prepare Document
In our model, each document represents the activity of a dealer on a day, which we called it Dealer_Day activity. In each document, there are tokens represented by a bond, or BOND_SYM_ID.

In [ ]:
# THIS TOO SLOW
# Add new column document to concatenate Report_Dealer_Index and TRD_RPT_DTTM
#data['document'] = data.apply(lambda x: str(x['Report_Dealer_Index'])+ ',' +str(x['TRD_RPT_DTTM'])[:10] ,axis=1)

In [ ]:
# This is how to get date of datetime series
#str(data['TRD_RPT_DTTM'].dt.date[0])

### Transform BOND_SYM_ID to BOND_SYM_ID_transformed with sklearn labelEncoder
Because both Gensim and SKlearn require token to be represented by interger ID, we have to encode current to interger ID.<br>
Not sure if this is needed for Gensim

In [ ]:
# Validate if the transform was correct
#data['BOND_SYM_ID_inversed_transformed'] = inverse_transform

In [ ]:
# make BOND_SYM_ID to dict ---- No need anymore

#BOND_SYM_ID_count_dict = data['BOND_SYM_ID'].value_counts().to_dict()
#keys = BOND_SYM_ID_count_dict.keys()
#np.array(list(keys))

#### Group data by ['document','BOND_SYM_ID_transformed'] so that represent a document and token inside it

In [ ]:
data_gb = data.groupby(by=['document','BOND_SYM_ID'])

In [ ]:
matrix_1 = data_gb['BOND_SYM_ID'].size().unstack(fill_value=0)
matrix_1 = matrix_1.sort_index(axis=1)

matrix_1_shape = matrix_1.shape
print('We have {} documents and {} tokens'.format(matrix_1_shape[0],matrix_1_shape[1]))

In [ ]:
root_folder = Path('../Data/Pickle/')
file_name = 'Matrix_1'
file_path = root_folder / file_name
matrix_1.to_pickle(file_path)

### Matrix 1 => Count of  BOND_SYM_ID in a document

#### Use SKlearn Label Encoder to transform tokens into integer

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(matrix_1.columns)
transform = le.transform(matrix_1.columns)
inverse_transform = le.inverse_transform(transform)

#### Create id2word to map label encoding to its words then create corpus with gensim Dense2Corpus which takes numpy

In [ ]:
id2word = dict(zip(transform, inverse_transform))

matrix1_corpus = gensim.matutils.Dense2Corpus(matrix_1.values,documents_columns=False)
# Run LDA in Gensim
lda = gensim.models.ldamulticore.LdaMulticore(corpus= matrix1_corpus,id2word=id2word,workers=3, num_topics=500, chunksize=10000, passes=1)

In [ ]:
# Save matrix_1 dictionary to file
#np.save('matrix_1.npy', matrix_1_dict['count']) 

### Matrix 2 => Sum of ENTRD_VOL_QT of a BOND_SYM_ID in document

In [ ]:
matrix_2 = data_gb[['ENTRD_VOL_QT']].sum()
matrix_2.head()

#### Transform Matrix 2 data 
https://scikit-learn.org/stable/modules/preprocessing.html

In [ ]:
matrix_2 = matrix_2.unstack(fill_value=0)
matrix_2.head()

### Matrix 3 => Product of ENTRD_VOL_QT and RPTD_PR of a BOND_SYM_ID in document
Create new column QT_X_PR as the product of ENTRD_VOL_QT and RPTD_PR<br>
Then do groupby QT_X_PR then sum them to get the total amount of the bond in that document

In [ ]:
data['QT_X_PR'] = data['ENTRD_VOL_QT'] * data['RPTD_PR']

In [ ]:
matrix_3 = data_gb[['QT_X_PR']].sum()
matrix_3.head()

In [ ]:
data['QT_X_PR'].describe()

In [ ]:
matrix_3 = matrix_3.unstack(fill_value=0)
matrix_3.head()

## Prepare Corpus for Gensim

### Output for SKlearn

## Data Validation

In [ ]:
shape = data.shape
print('We have {} rows {} columns'.format(shape[0],shape[1]))

In [ ]:
n_duplicated = data.duplicated().sum()
percentage = n_duplicated/shape[0]*100
print('{} rows that are entirely the same in the data set which is {:.2f}'.format(n_duplicated,percentage))

In [ ]:
print('Number of duplcations based on grouping keys:')
test_duplication = [['BOND_SYM_ID'],['CUSIP_ID'],['BOND_SYM_ID','CUSIP_ID'],['Report_Dealer_Index','TRD_EXCTN_DTTM'],
                   ['Report_Dealer_Index','TRD_EXCTN_DTTM','BOND_SYM_ID']]
for test in test_duplication:
    print('{} : {}'.format(test, data.duplicated(subset=test,keep='first').sum()))

In [ ]:
data.loc[data.duplicated(keep=False,subset=['Report_Dealer_Index','TRD_EXCTN_DTTM','BOND_SYM_ID'])].sort_values(by=['BOND_SYM_ID'])